In [1]:
import os
import librosa
import numpy as np


In [2]:
def look_through(category):
    
    listOfFiles = os.listdir(category)
    arrays_sound = np.empty((0,193))
    for file in listOfFiles:
        filename = os.path.join(category,file)
        features_sound = extract_data(filename)
        arrays_sound = np.vstack((arrays_sound,features_sound))
    return arrays_sound

In [3]:
def extract_data(file_name):
    
    X, sample_rate = librosa.load(file_name , duration = 5.0)
    stft = np.abs(librosa.stft(X))
    mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T,axis=0)
    chroma = np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)
    mel = np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
    contrast = np.mean(librosa.feature.spectral_contrast(S=stft, sr=sample_rate).T,axis=0)
    tonnetz = np.mean(librosa.feature.tonnetz(y=librosa.effects.harmonic(X),sr=sample_rate).T,axis=0)
    return np.hstack((mfccs,chroma,mel,contrast,tonnetz))

SPEAKER 1 SOUNDS We are assigning 0 to speaker one as the numbe rof speakers incrfease we have to assign each of them a label

In [ ]:
speaker1_sounds = look_through('Insert folder which has all sounds for speaker 1')
speaker1_labels = [0 for items in speaker_sounds]

In [ ]:
speaker2_sounds = look_through('Insert folder for all sounds of speaker 2')
speaker2_labels = [1 for items in speaker2_sounds]

As the number of speakers increase we add a new label for each speaker

In [ ]:
train_sounds = np.concatenate((speaker1_sounds, speaker2_sounds))
train_labels = np.concatenate((speaker1_labels, speaker2_labels))

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split, cross_val_score

X_train, X_test, y_train, y_test = train_test_split(
    train_sounds, train_labels, test_size=0.2, random_state=42
)

In [6]:
import xgboost as xgb

In [ ]:
params = {
    
    'max_depth': 8, 
    'eta':.03,
    'subsample': 1, 
    'colsample_bytree': 0.8, 
    'n_estimators': 1000,
    'objective':'binary:logistic',
    'eval_metric':'logloss',
    'silent': 1
}

Experiment with the params to get even better results as it is a binary classification model the objective is binary:logistic refer to xgboost documentation to change objective if there are more than 2 speakers

In [ ]:
dt = xgdmat=xgb.DMatrix(X_train,y_train)

In [ ]:
dv = xgdmat=xgb.DMatrix(X_test,y_test)

In [ ]:
model = xgb.train(params, dt, 400, [(dt, "train"),(dv, "valid")], verbose_eval=200)

experiment with the value 400 increaseing and decreasing to get better results stop experimenting when the logloss starts increasing

In [7]:
from sklearn.metrics import confusion_matrix

In [8]:
from termcolor import colored

In [ ]:
y_pred = model.predict(dv)
# Making the Confusion Matrix
cm = confusion_matrix(y_test, (y_pred>0.5))
print(colored('The Confusion Matrix is: ', 'red'),'\n', cm)
# Calculate the accuracy on test set
predict_accuracy_on_test_set = (cm[0,0] + cm[1,1])/(cm[0,0] + cm[1,1]+cm[1,0] + cm[0,1])
print(colored('The Accuracy on Test Set is: ', 'blue'), colored(predict_accuracy_on_test_set, 'blue'))

ABOVE snippet predicts accuracy and shows the confusion matrix